In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report
from textblob import TextBlob
from nltk.tokenize import word_tokenize
!pip install contractions
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.metrics import classification_report
from textblob import TextBlob
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to /Users/gopi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data parsing

In [51]:
df = pd.read_csv("data.tsv", usecols = ['review_body', 'star_rating'], sep='\t')
df['star_rating'] =  pd.to_numeric(df['star_rating'], errors='coerce')
df = df.dropna(subset = ['star_rating'])
df.loc[:,'star_rating'] = df['star_rating'].astype(int)
df.loc[:,'review_body'] = df['review_body'].astype(str)

data_set = pd.DataFrame()
for i in range(1,6):
    temp = df[df['star_rating'] == i].sample(frac=1).reset_index(drop=True).iloc[:20000]
    data_set =data_set.append(temp, ignore_index=True)   
data_set = data_set.sample(frac=1).reset_index(drop=True)

/opt/homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Question 2

In [40]:
import gensim.downloader as gensim
google_w2v = gensim.load('word2vec-google-news-300')

In [41]:
word_pairs = [('pocket', 'chain'), ('man','boy'),('good','best'),('beads','necklace'),('ring','diamond'),('excellent', 'outstanding'), ('woman','girl')]
for w1,w2 in word_pairs:
    print('%r\t%r\t%.2f' % (w1, w2, google_w2v.similarity(w1, w2)))

'pocket'	'chain'	-0.03
'man'	'boy'	0.68
'good'	'best'	0.55
'beads'	'necklace'	0.56
'ring'	'diamond'	0.29
'excellent'	'outstanding'	0.56
'woman'	'girl'	0.75


# Basic pre processing

In [52]:
import time
#converting reviews to lower cases
data_set['review_body'] = data_set['review_body'].str.lower()
#extracting content in html tags
data_set['review_body'] = data_set['review_body'].apply(lambda x : BeautifulSoup(x).get_text())
#removing all urls in the review
data_set['review_body'] = data_set['review_body'].apply(lambda x : re.sub(r'http\S+','',x).strip())
#expanding contraction words
data_set['review_body'] = data_set['review_body'].apply(lambda x : contractions.fix(x))
#removing non alphabetical characters
data_set['review_body'] = data_set['review_body'].apply(lambda x : " ".join(re.sub('[^a-z]+','', word) for word in word_tokenize(x)))
#removing extra spaces between words
data_set['review_body'] = data_set['review_body'].apply(lambda x : re.sub(' +',' ',x).strip())
data_set['review_list'] = data_set['review_body'].apply(lambda x : x.split(" "))

/opt/homebrew/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [43]:
import gensim.models
word2vec = gensim.models.Word2Vec(sentences=data_set['review_list'], vector_size=300, min_count=11)

In [44]:
word_pairs = [('pocket', 'chain'), ('man','boy'),('good','best'),('beads','necklace'),('ring','diamond'), ('excellent', 'outstanding'),('woman','girl')]
for w1,w2 in word_pairs:
    print('%r\t%r\t%.2f' % (w1, w2, word2vec.wv.similarity(w1,w2)))
    

'pocket'	'chain'	0.07
'man'	'boy'	0.48
'good'	'best'	0.38
'beads'	'necklace'	0.23
'ring'	'diamond'	0.27
'excellent'	'outstanding'	0.71
'woman'	'girl'	0.72


In [45]:
from numpy.linalg import norm
king = google_w2v['king']
man = google_w2v['man']
woman = google_w2v['woman']
resulting_vec = king-man+woman
new_vec = google_w2v['queen'] - woman + man
beads= google_w2v['beads']
thread = google_w2v['thread']
necklace_vec = beads + thread
print("semantic similarity between king - man + woman = king is :",np.dot(resulting_vec, google_w2v['queen'])/(norm(resulting_vec)*norm(google_w2v['queen'])))
print("semantic similarity between queen - woman + man = king is :",np.dot(new_vec, google_w2v['king'])/(norm(new_vec)*norm(google_w2v['king'])))
print("semantic similarity between beads + thread = necklace is :",np.dot(necklace_vec, google_w2v['necklace'])/(norm(necklace_vec)*norm(google_w2v['necklace'])))

semantic similarity between king - man + woman = king is : 0.7300517
semantic similarity between queen - woman + man = king is : 0.70464075
semantic similarity between beads + thread = necklace is : 0.5135838


In [46]:
from numpy.linalg import norm
king = word2vec.wv['king']
man = word2vec.wv['man']
woman = word2vec.wv['woman']
resulting_vec = king-man+woman
new_vec = word2vec.wv['queen'] - woman + man
beads= word2vec.wv['beads']
thread = word2vec.wv['thread']
necklace_vec = beads + thread
print("semantic similarity between king - man + woman = king is :",np.dot(resulting_vec, word2vec.wv['queen'])/(norm(resulting_vec)*norm(word2vec.wv['queen'])))
print("semantic similarity between queen - woman + man = king is :",np.dot(new_vec, word2vec.wv['king'])/(norm(new_vec)*norm(word2vec.wv['king'])))
print("semantic similarity between beads + thread = necklace is :",np.dot(necklace_vec, word2vec.wv['necklace'])/(norm(necklace_vec)*norm(word2vec.wv['necklace'])))

semantic similarity between king - man + woman = king is : 0.29035768
semantic similarity between queen - woman + man = king is : -0.050481144
semantic similarity between beads + thread = necklace is : 0.2400201


In [56]:
#from gensim.models import KeyedVectors
#word_vectors = word_2_vec
#word_vectors.save("word2vec.wordvectors")
#google_w2v = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [10]:
def print_metrics(report):
  a = "accuracy is: " + str(report['accuracy']) + "\n"
  p = "precision is: " + str(report['macro avg']['precision']) + "\n"
  r = "recall is: " + str(report['macro avg']['recall']) + "\n"
  f1 = "F1 score is: " + str(report['macro avg']['f1-score']) + "\n"
  return p+r+f1+a

# Question 3:

In [53]:
global_set = []
def get_avg_word2vec(x):
  #print(x)
  count = 0
  add_vector = np.zeros(300)
  for word in x:
        if word in google_w2v:
            add_vector = np.add(add_vector, google_w2v[word])
            count += 1
  if count > 0:
    add_vector = np.divide(add_vector, len(x))
  global_set.append(add_vector)   
  return add_vector
    

data_set['review_list'].apply(lambda x: get_avg_word2vec(x))
#data_set.head(5)

0        [0.019232177734375, 0.04854736328125, 0.044506...
1        [-0.0034208943073014567, 0.027732433233045993,...
2        [0.12060546875, 0.118896484375, -0.05334472656...
3        [0.020255088806152344, 0.036087870597839355, 0...
4        [-0.000396728515625, -7.926093207465278e-05, 0...
                               ...                        
99995    [-0.04798719618055555, 0.003268771701388889, 0...
99996    [0.1359375, -0.027951431274414063, -0.04521484...
99997    [0.026754483309659093, 0.013201210715553977, 0...
99998    [0.001480712890625, 0.01921142578125, 0.013985...
99999    [0.028919677734375, 0.04644935607910156, 0.070...
Name: review_list, Length: 100000, dtype: object

In [54]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(global_set, data_set['star_rating'], test_size = 0.2)


# Perceptron Model

In [55]:
from sklearn.linear_model import Perceptron
perceptron_model = Perceptron()
perceptron_model.fit(x_train, y_train)
y_test_pred_perceptron = perceptron_model.predict(x_test)
report = classification_report(y_test, y_test_pred_perceptron,output_dict=True)
print(print_metrics(report))

precision is: 0.467763105616742
recall is: 0.3737731280706565
F1 score is: 0.3371223090952351
accuracy is: 0.37155



# Linear SVM Model

In [50]:
from sklearn.svm import LinearSVC
svm_classifier = LinearSVC()
svm_classifier.fit(x_train, y_train)
y_test_pred_svm = svm_classifier.predict(x_test)
report=classification_report(y_test, y_test_pred_svm,output_dict=True)
print(print_metrics(report))

precision is: 0.4651957853936747
recall is: 0.48321042971229333
F1 score is: 0.46348707325528676
accuracy is: 0.48505



# Question 4a: MLP with average word vectors

In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as Functions

class LoadDataset(Dataset):
  def __init__(self, data):
    self.x_train = torch.tensor(data[0], dtype = torch.float32)
    self.y_train = torch.tensor(data[1].to_numpy(), dtype = torch.long)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self, idx):
    return self.x_train[idx],self.y_train[idx]

In [23]:
input_nodes = 300
hidden_1 = 50
hidden_2 = 10
output = 5

class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()

    self.layer_1 = nn.Linear(input_nodes, hidden_1)
    self.layer_2 = nn.Linear(hidden_1, hidden_2)
    self.layer_3 = nn.Linear(hidden_2, output)

    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    #x = x.view(-1, input)
    x = Functions.relu(self.layer_1(x))
    x = self.dropout(x)
    x = Functions.relu(self.layer_2(x))
    x = self.dropout(x)
    x = self.layer_3(x)
    return x

  def set_dropout(self, dp):
        self.dropout = nn.Dropout(dp)

mlp_model = NeuralNet()
#print(mlp_model)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=0.01)

In [24]:
def train_validation_split(x_train, y_train):
  x_validation = x_train[0:int(0.2*len(x_train))]
  y_validation = y_train[0:int(0.2*len(y_train))]
  x_train = x_train[int(0.2*len(x_train)):len(x_train)]
  y_train = y_train[int(0.2*len(y_train)):len(y_train)]
  return x_train, y_train, x_validation, y_validation

def train_network(model, train_loader, validation_loader, n_epochs):
  validation_loss_min = np.Inf
  for epoch in range(n_epochs):
    train_loss = 0.0
    validation_loss = 0.0
    model.train()

    for index, (data,label) in enumerate(train_loader):
      optimizer.zero_grad()
      output = model(data)
      loss = criteria(output, label-1)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*data.size(0)
  
    model.eval()
    for index, (data, label) in enumerate(validation_loader):
      output = model(data)
      loss = criteria(output, label-1)
      validation_loss += loss.item()*data.size(0)

    
    train_loss = train_loss/len(train_loader.dataset)
    validation_loss = validation_loss/len(validation_loader.dataset)
    if validation_loss <= validation_loss_min:
        torch.save(model.state_dict(), 'model.pt')
        validation_loss_min = validation_loss  

x_train, y_train, x_validation, y_validation = train_validation_split(x_train, y_train)

train_dataset = LoadDataset((x_train, y_train))
validation_dataset = LoadDataset((x_validation, y_validation))

/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/1734367578.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  self.x_train = torch.tensor(data[0], dtype = torch.float32)


In [25]:
batch_size = 50
mlp_train_loader = DataLoader(train_dataset, batch_size=batch_size)
mlp_validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
train_network(mlp_model, mlp_train_loader, mlp_validation_loader, 100)

In [26]:
mlp_model.load_state_dict(torch.load('model.pt'))
test_dataset = LoadDataset((x_test,y_test))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

def predict(model, dataloader):
    prediction_list = []
    for index, (data, label) in enumerate(dataloader):
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions = predict(mlp_model,test_loader)
predictions = np.array(predictions)
predictions_int = []
for pred in predictions:
  predictions_int = np.append(predictions_int, int(pred) + 1)
report=classification_report(y_test, predictions_int,output_dict=True)
print(print_metrics(report))

precision is: 0.49004611709622187
recall is: 0.5026240744646133
F1 score is: 0.4917581448876289
accuracy is: 0.5032



/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/3416720493.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  predictions = np.array(predictions)
/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/3416720493.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(predictions)


# Question 4b: MLP with concatenated word vectors

In [35]:
mlp_vectors_concat = []
def get_vectors_concat(words):
  result = []
  count = 0
  for word in words:
    if word in google_w2v and count < 10:
      result.extend(google_w2v[word])
      count+=1
    if count > 10:
        break

  diff = 3000 - len(result)
  result.extend(np.zeros(diff))
  mlp_vectors_concat.append(result)
  return result

data_set['review_list'].apply(lambda x: get_vectors_concat(x))
#data_set['review_vectors'] = data_set['review_vectors'].apply(lambda x : np.array(x))
#print(data_set.head(5))

0        [0.084472656, -0.0003528595, 0.053222656, 0.09...
1        [0.10107422, -0.0038146973, 0.018188477, 0.129...
2        [-0.22558594, -0.01953125, 0.09082031, 0.23730...
3        [0.080078125, 0.10498047, 0.049804688, 0.05346...
4        [0.084472656, -0.0003528595, 0.053222656, 0.09...
                               ...                        
99995    [-0.016235352, 0.09423828, 0.091796875, 0.1196...
99996    [0.109375, 0.140625, -0.03173828, 0.16601562, ...
99997    [0.17089844, 0.024291992, 0.13867188, 0.022216...
99998    [0.109375, 0.140625, -0.03173828, 0.16601562, ...
99999    [0.14453125, 0.04711914, 0.10058594, 0.328125,...
Name: review_list, Length: 100000, dtype: object

In [36]:
batch_size = 100
x_train,x_test,y_train,y_test = train_test_split(mlp_vectors_concat, data_set['star_rating'], test_size = 0.2)
x_train, y_train, x_validation, y_validation = train_validation_split(x_train, y_train)
#print(len(x_train))
train_dataset = LoadDataset((x_train, y_train))
validation_dataset = LoadDataset((x_validation, y_validation))
test_dataset = LoadDataset((x_test,y_test))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle = True)

In [37]:
input_nodes = 3000
hidden_1 = 50
hidden_2 = 10
output = 5

mlp_concat_model = NeuralNet()
mlp_concat_model.set_dropout(0.5)
#print(mlp_concat_model)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_concat_model.parameters(), lr=1e-4, weight_decay=1e-5)
train_network(mlp_concat_model, train_loader, validation_loader, 50)

In [38]:
def predict(model, dataloader):
    prediction_list = []
    for index, (data, label) in enumerate(dataloader):
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions = predict(mlp_concat_model,test_loader)
predictions = np.array(predictions)
predictions_int = []
for pred in predictions:
  predictions_int = np.append(predictions_int, int(pred) + 1)
report=classification_report(y_test, predictions_int,output_dict=True)
print(print_metrics(report))

precision is: 0.4106313941791613
recall is: 0.4278203623274702
F1 score is: 0.4108242828066257
accuracy is: 0.4282



/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2232472655.py:10: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  predictions = np.array(predictions)
/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2232472655.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(predictions)


# Question 5a: Basic RNN

In [14]:
global_rnn_vectors = []
def get_vectors(words):
  result = []
  count = 0
  for word in words:
    if word in google_w2v and count < 20:
      result.extend(google_w2v[word])
      count+=1
    if count > 20:
        break

  diff = 6000 - len(result)
  result.extend(np.zeros(diff))
  global_rnn_vectors.append(result)
  return result
data_set['review_list'].apply(lambda x: get_vectors(x))

0        [0.084472656, -0.0003528595, 0.053222656, 0.09...
1        [0.10107422, -0.0038146973, 0.018188477, 0.129...
2        [-0.22558594, -0.01953125, 0.09082031, 0.23730...
3        [0.080078125, 0.10498047, 0.049804688, 0.05346...
4        [0.084472656, -0.0003528595, 0.053222656, 0.09...
                               ...                        
99995    [-0.016235352, 0.09423828, 0.091796875, 0.1196...
99996    [0.109375, 0.140625, -0.03173828, 0.16601562, ...
99997    [0.17089844, 0.024291992, 0.13867188, 0.022216...
99998    [0.109375, 0.140625, -0.03173828, 0.16601562, ...
99999    [0.14453125, 0.04711914, 0.10058594, 0.328125,...
Name: review_list, Length: 100000, dtype: object

In [15]:
count = 0
for x in global_rnn_vectors:
    if len(x) != 6000:
        count+=1
print(count)

0


In [27]:
x_train,x_test,y_train,y_test = train_test_split(global_rnn_vectors, data_set['star_rating'], test_size = 0.2)
x_train, y_train, x_validation, y_validation = train_validation_split(x_train, y_train)

train_dataset = LoadDataset((x_train, y_train))
validation_dataset = LoadDataset((x_validation, y_validation))
test_dataset = LoadDataset((x_test,y_test))

train_loader = DataLoader(train_dataset, batch_size=100, shuffle = True)
validation_loader = DataLoader(validation_dataset, batch_size=100, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

In [28]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as Functions


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)

        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out,_ = self.rnn(x,h0)
        out = out[:,-1,:]
        out = self.fc(out)
        return out
    

n_hidden = 20
rnn_model = RNN(300, 20, 2, 5)
print(rnn_model)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn_model.parameters(), lr=0.01)


RNN(
  (rnn): RNN(300, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)


In [29]:
n_total_steps = len(train_loader)

n_epochs = 100
validation_loss_min = np.Inf
for epoch in range(n_epochs):
  train_loss = 0.0
  validation_loss = 0.0
  for index, (data,label) in enumerate(train_loader):
    data = data.reshape(-1, 20, 300)
    #print(data.shape)
    outputs = rnn_model(data)
    
    loss = criteria(outputs, label-1)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  
    rnn_model.eval()
  for index, (data, label) in enumerate(validation_loader):
      data = data.reshape(-1, 20, 300)
      output = rnn_model(data)
      loss = criteria(output, label-1)
      validation_loss += loss.item()*data.size(0)

    
  train_loss = train_loss/len(train_loader.dataset)
  validation_loss = validation_loss/len(validation_loader.dataset)
  if validation_loss <= validation_loss_min:
        torch.save(rnn_model.state_dict(), 'rnn_model.pt')
        validation_loss_min = validation_loss  

In [30]:
rnn_model.load_state_dict(torch.load('rnn_model.pt'))
rnn_sample_pred = []
with torch.no_grad():
  for index, (data,label) in enumerate(test_loader):
    data = data.reshape(-1, 20, 300)
    #print(data.shape)
    outputs = rnn_model(data)

    _,predicted = torch.max(outputs.data, 1)

    rnn_sample_pred.append(predicted)


rnn_predictions = np.array(rnn_sample_pred)
predictions_int = []
for pred in rnn_predictions:
  predictions_int = np.append(predictions_int, int(pred) + 1)
report=classification_report(y_test, predictions_int,output_dict=True)
print(print_metrics(report))



precision is: 0.45601098554727565
recall is: 0.4732367548006525
F1 score is: 0.45753562317164176
accuracy is: 0.47115



/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2842718222.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  rnn_predictions = np.array(rnn_sample_pred)
/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2842718222.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rnn_predictions = np.array(rnn_sample_pred)


# Question 5b: Gated RNN

In [31]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as Functions


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRU, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first = True)

        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out,_ = self.rnn(x,h0)
        out = out[:,-1,:]
        out = self.fc(out)
        return out
    

n_hidden = 20
gru_model = GRU(300, 20, 2, 5)
print(gru_model)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(gru_model.parameters(), lr=0.1)


GRU(
  (rnn): GRU(300, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)


In [32]:
x_train,x_test,y_train,y_test = train_test_split(global_rnn_vectors, data_set['star_rating'], test_size = 0.2)
x_train, y_train, x_validation, y_validation = train_validation_split(x_train, y_train)

train_dataset = LoadDataset((x_train, y_train))
validation_dataset = LoadDataset((x_validation, y_validation))
test_dataset = LoadDataset((x_test,y_test))

train_loader = DataLoader(train_dataset, batch_size=50, shuffle = True)
validation_loader = DataLoader(validation_dataset, batch_size=50, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

In [33]:
import time
start_time = time.time()
n_total_steps = len(train_loader)

n_epochs = 50
validation_loss_min = np.Inf
for epoch in range(n_epochs):
  train_loss = 0.0
  validation_loss = 0.0
  for index, (data,label) in enumerate(train_loader):
    data = data.reshape(-1, 20, 300)
    #print(data.shape)
    outputs = gru_model(data)
    
    loss = criteria(outputs, label-1)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  
    gru_model.eval()
  for index, (data, label) in enumerate(validation_loader):
      data = data.reshape(-1, 20, 300)
      output = gru_model(data)
      loss = criteria(output, label-1)
      validation_loss += loss.item()*data.size(0)

    
  train_loss = train_loss/len(train_loader.dataset)
  validation_loss = validation_loss/len(validation_loader.dataset)
    
  if validation_loss <= validation_loss_min:
        torch.save(gru_model.state_dict(), 'gru_model.pt')
        validation_loss_min = validation_loss  

print("Time taken is:", time.time() - start_time)

Time taken is: 380.6278429031372


In [34]:
gru_model.load_state_dict(torch.load('gru_model.pt'))
rnn_sample_pred = []
with torch.no_grad():
  for index, (data,label) in enumerate(test_loader):
    data = data.reshape(-1, 20, 300)
    #print(data.shape)
    outputs = gru_model(data)

    _,predicted = torch.max(outputs.data, 1)

    rnn_sample_pred.append(predicted)


rnn_predictions = np.array(rnn_sample_pred)
predictions_int = []
for pred in rnn_predictions:
  predictions_int = np.append(predictions_int, int(pred) + 1)
report=classification_report(y_test, predictions_int,output_dict=True)
print(print_metrics(report))



precision is: 0.49368513424744964
recall is: 0.5027778621641076
F1 score is: 0.4957633845981797
accuracy is: 0.50255



/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2153500255.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  rnn_predictions = np.array(rnn_sample_pred)
/var/folders/y9/k2kkzqkj49jcv8lx3vwms0140000gn/T/ipykernel_676/2153500255.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rnn_predictions = np.array(rnn_sample_pred)
